# Подготовка данных

In [ ]:
import pandas as pd
import numpy as np
from sklearn import model_selection, metrics, tree, ensemble

In [ ]:
df = pd.read_csv('weather_preprocessed.csv', index_col=0)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
# Разделение признаков 
y = df['RainTomorrow'].values
df.pop('RainTomorrow')
X = df.values

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Обучение композиций

## Бэггинг

### Модель дерева решений

#### Обучение модели дерева решений

In [ ]:
tree_class = tree.DecisionTreeClassifier()

In [ ]:
%%time
tree_class.fit(X_train, y_train)

#### Подбор гиперпараметров для модели дерева решений

In [ ]:
# Текущие значения гиперпараметров у модели
tree_class.get_params()

In [ ]:
# подбираемые гиперпараметры
params = {
    'criterion' : ['gini', 'entropy'],
    'splitter' : ['best', 'random'],
    'min_samples_split' : [2, 4, 6, 8, 10], 
}

In [ ]:
# Создание модели GridSearchCV
grid_tree_class = model_selection.GridSearchCV(tree.DecisionTreeClassifier(), params)

In [ ]:
%%time
# Обучение модели
grid_tree_class.fit(X_train, y_train)

In [ ]:
# Оптимальная модель
grid_tree_class.best_estimator_

### Композиция без гиперпаметров

#### Обучение композиции

In [ ]:
# Создание модели
bag_class = ensemble.BaggingClassifier(tree.DecisionTreeClassifier())

In [ ]:
%%time
# Обучение модели
bag_class.fit(X_train, y_train)

#### Оценка качества

##### Качество стандартной модели дерева решений

In [ ]:
y_pred = tree_class.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

##### Качество модели дерева решений с подобранными гиперпараметрами

In [ ]:
y_pred = grid_tree_class.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

##### Качество композиции

In [ ]:
y_pred = bag_class.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

Качество у модели композиции значительно выше, чем у базовой модели.

Время работы стандартной модели дерева решений - 4,62s

Время работы модели дерева решений с подобранными гиперпараметрами

Время работы композиции без гиперпараметров - 33,9s

### Композиция с гиперпараметрами у базовых алгоритмов

#### Обучение композиции

In [ ]:
# Создание композиции из моделей дерева решений
bag_class = ensemble.BaggingClassifier(tree.DecisionTreeClassifier(min_samples_split=10, splitter='random'))

In [ ]:
%%time
# Обучение композиции
bag_class.fit(X_train, y_train)

#### Оценка качества

##### Качество стандартной модели дерева решений

In [ ]:
y_pred = tree_class.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

##### Качество модели дерева решений с подобранными гиперпараметрами

In [ ]:
y_pred = grid_tree_class.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

##### Качество композиции

In [ ]:
y_pred = bag_class.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

Качество у модели композиции значительно выше, чем у базовой модели.

Время работы стандартной модели дерева решений - 6 min 28 s

Время работы модели дерева решений с подобранными гиперпараметрами - 

Время работы композиции с гиперпараметрами у базовых моделей - 35.8 s

### Композиция с гиперпараметрами

#### Подбор гиперпараметров для композиции

In [ ]:
# Стандартная композиция, состоящая из моделей решеющих деревьев
bag_class = ensemble.BaggingClassifier(tree.DecisionTreeClassifier())

In [ ]:
# Текущие значения гиперпараметров композиции
bag_class.get_params()

In [ ]:
# Подбираемые гиперпараметры
params = {
    'n_estimators' : [10, 15, 20, 25],
}

In [ ]:
# Создание модели GridSearchCV
grid_bag_class = model_selection.GridSearchCV(bag_class, params)

In [ ]:
%%time
# Обучение модели
grid_bag_class.fit(X_train, y_train)

In [ ]:
# Оптимальная модель
grid_bag_class.best_estimator_

#### Оценка качества

##### Качество стандартной модели дерева решений

In [ ]:
y_pred = tree_class.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

##### Качество модели дерева решений с подобранными гиперпараметрами

In [ ]:
y_pred = grid_tree_class.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

##### Качество композиции

In [ ]:
y_pred = grid_bag_class.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

Качество модели композиции значительно выше, чем у базовой модели.

Время работы стандартной модели дерева решений - 4.33 s

Время работы модели дерева решений с подобранными гиперпараметрами 

Время работы композиции c подобранными гиперпараметрами - 16min 59s

### Композиция с гиперпараметрами и с гиперпараметрами у базовых моделей

#### Обучение композиции

In [ ]:
bag_class = ensemble.BaggingClassifier(tree.DecisionTreeClassifier(min_samples_split=10, splitter='random'), n_estimators=20)

In [ ]:
%%time
bag_class.fit(X_train, y_train)

#### Оценка качества

##### Качество базовой модели

In [ ]:
y_pred = tree_class.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

##### Качество модели дерева решений с подобранными гиперпараметрами

In [ ]:
y_pred = grid_tree_class.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

##### Качество композиции

In [ ]:
y_pred = bag_class.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

Качество модели композиции значительно выше, чем у базовой модели.

Время работы базовой модели - 3.1 s

Время работы модели дерева решений с подобранными гиперпараметрами

Время работы композиции - 50.1 s

## Бустинг

### Модель дерева решений

#### Обучение модели дерева решений

In [ ]:
tree_class = tree.DecisionTreeClassifier()

In [ ]:
%%time
tree_class.fit(X_train, y_train)

#### Подбор гиперпараметров для модели дерева решений

In [ ]:
# Текущие значения гиперпараметров у модели
tree_class.get_params()

In [ ]:
# подбираемые гиперпараметры
params = {
    'criterion' : ['gini', 'entropy'],
    'splitter' : ['best', 'random'],
    'min_samples_split' : [2, 4, 6, 8, 10], 
}

In [ ]:
# Создание модели GridSearchCV
grid_tree_class = model_selection.GridSearchCV(tree.DecisionTreeClassifier(), params)

In [ ]:
%%time
# Обучение модели
grid_tree_class.fit(X_train, y_train)

In [ ]:
# Оптимальная модель
grid_tree_class.best_estimator_

### Композиция без гиперпараметров

#### Обучение композиции

In [ ]:
grad_boost_class = ensemble.GradientBoostingClassifier()

In [ ]:
%%time
grad_boost_class.fit(X_train, y_train)

#### Оценка качества

##### Качество стандартной модели дерева решений

In [ ]:
y_pred = tree_class.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

##### Качество модели дерева решений с подобранными гиперпараметрами

In [ ]:
y_pred = grid_tree_class.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

##### Качество композиции

In [ ]:
y_pred = grad_boost_class.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

Качество модели композиции значительно выше, чем у модели дерева решений.

Время работы стандартной модели дерева решений - 4.26 s

Время работы композиции - 1min 49s

### Композиция с гиперпараметрами

In [ ]:
grad_boost_class = ensemble.GradientBoostingClassifier()

In [ ]:
grad_boost_class.get_params()

In [ ]:
params = {
    'ccp_alpha' : [0.0, 0.1, 0.2],
    'max_depth' : [3, 4, 5]
}

In [ ]:
grid_grad_boost_class = model_selection.GridSearchCV(grad_boost_class, params)

In [ ]:
%%time
# Обучение модели
grid_grad_boost_class.fit(X_train, y_train)

In [ ]:
# Оптимальная модель
grid_tree_class.best_estimator_

#### Оценка качества

##### Качество стандартной модели дерева решений

In [ ]:
y_pred = tree_class.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

##### Качество модели дерева решений с подобранными гиперпараметрами

In [ ]:
y_pred = grid_tree_class.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

##### Качество композиции

In [ ]:
y_pred = grid_grad_boost_class.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

## Стекинг

### Композиция без гиперпараметров

In [ ]:
stack_class = ensemble.StackingClassifier([('dtc', tree.DecisionTreeClassifier())])

In [ ]:
%%time
stack_class.fit(X_train, y_train)

### Композиция с гиперпараметрами у базовых моделей

In [ ]:
stack_class = ensemble.StackingClassifier([('dtc', tree.DecisionTreeClassifier(min_samples_split=10, splitter='random'))])

In [ ]:
%%time
stack_class.fit(X_train, y_train)

### Композиция с гиперпараметрами

In [ ]:
stack_class = ensemble.StackingClassifier([('dtc', tree.DecisionTreeClassifier())])

In [ ]:
stack_class.get_params()

In [ ]:
params = {
    'stack_method' : ['auto', 'predict_proba']
}

In [ ]:
grid_stack_class = model_selection.GridSearchCV(stack_class, params)

In [ ]:
%%time
grid_stack_class.fit(X_train, y_train)

In [ ]:
grid_stack_class.best_estimator_

### Композиция с гиперпараметрами и с гиперпараметрами у базовых моделей

In [ ]:
stack_class = ensemble.StackingClassifier([('dtc', tree.DecisionTreeClassifier(min_samples_split=10, splitter='random'))], stack_method='predict_proba')

In [ ]:
%%time
stack_class.fit(X_train, y_train)